In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict

In [213]:
curr_dir = !pwd
datafile_date = "08.06.18"
prec_th_str = "dna0.5_rna0.25_ion0.75"

input_path = curr_dir[0]+"/phase1_initial_run/"+datafile_date+"_"+prec_th_str+"/per_trial/"
output_path = curr_dir[0]+"/phase1_initial_run/"+datafile_date+"_"+prec_th_str+"/"

#ligands = ["dna", "dnabase", "dnabackbone", "rna", "rnabase", "rnabackbone", "peptide", "ion", "metabolite", "all_ligands"]
#ligands = ["metabolite", "peptide", "rna", "ion", "sm", "dna"]
ligands = ["sm"]
#classifiers = ["RF", "SVM", "KNN", "ADA", "XGB"]
classifiers = ["SVM"]
no_trials = 100
folds_num = 5


### Copy relevant files from the cluster

In [207]:
#Remember to change the folders in both src and dest if need be!
!scp -r -p anatf@gen-comp1:/Genomics/grid/users/anatf/ExAC/10.Prediction/hyperparam_tuning/phase1_initial_run/08.06.18_dna0.5_rna0.25_ion0.75/per_trial/sm_ADA* /home/anat/Research/ExAC/10.Prediction/hyperparam_tuning/phase1_initial_run/08.06.18_dna0.5_rna0.25_ion0.75/per_trial/.

sm_ADA_fold1_trial0_5w_hyperparameters.csv    100%  205     0.2KB/s   00:01    
sm_ADA_fold1_trial10_5w_hyperparameters.csv   100%  205     0.2KB/s   00:00    
sm_ADA_fold1_trial11_5w_hyperparameters.csv   100%  204     0.2KB/s   00:00    
sm_ADA_fold1_trial12_5w_hyperparameters.csv   100%  209     0.2KB/s   00:00    
sm_ADA_fold1_trial13_5w_hyperparameters.csv   100%  202     0.2KB/s   00:00    
sm_ADA_fold1_trial14_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold1_trial15_5w_hyperparameters.csv   100%  207     0.2KB/s   00:00    
sm_ADA_fold1_trial1_5w_hyperparameters.csv    100%  208     0.2KB/s   00:00    
sm_ADA_fold1_trial16_5w_hyperparameters.csv   100%  209     0.2KB/s   00:00    
sm_ADA_fold1_trial17_5w_hyperparameters.csv   100%  207     0.2KB/s   00:00    
sm_ADA_fold1_trial18_5w_hyperparameters.csv   100%  205     0.2KB/s   00:00    
sm_ADA_fold1_trial19_5w_hyperparameters.csv   100%  209     0.2KB/s   00:00    
sm_ADA_fold1_trial20_5w_hyperparameters.

sm_ADA_fold2_trial14_5w_hyperparameters.csv   100%  207     0.2KB/s   00:00    
sm_ADA_fold2_trial15_5w_hyperparameters.csv   100%  209     0.2KB/s   00:01    
sm_ADA_fold2_trial1_5w_hyperparameters.csv    100%  208     0.2KB/s   00:00    
sm_ADA_fold2_trial16_5w_hyperparameters.csv   100%  207     0.2KB/s   00:00    
sm_ADA_fold2_trial17_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold2_trial18_5w_hyperparameters.csv   100%  205     0.2KB/s   00:00    
sm_ADA_fold2_trial19_5w_hyperparameters.csv   100%  210     0.2KB/s   00:00    
sm_ADA_fold2_trial20_5w_hyperparameters.csv   100%  209     0.2KB/s   00:00    
sm_ADA_fold2_trial21_5w_hyperparameters.csv   100%  210     0.2KB/s   00:00    
sm_ADA_fold2_trial22_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold2_trial23_5w_hyperparameters.csv   100%  205     0.2KB/s   00:00    
sm_ADA_fold2_trial24_5w_hyperparameters.csv   100%  205     0.2KB/s   00:00    
sm_ADA_fold2_trial25_5w_hyperparameters.

sm_ADA_fold3_trial1_5w_hyperparameters.csv    100%  206     0.2KB/s   00:00    
sm_ADA_fold3_trial16_5w_hyperparameters.csv   100%  206     0.2KB/s   00:00    
sm_ADA_fold3_trial17_5w_hyperparameters.csv   100%  206     0.2KB/s   00:00    
sm_ADA_fold3_trial18_5w_hyperparameters.csv   100%  207     0.2KB/s   00:00    
sm_ADA_fold3_trial19_5w_hyperparameters.csv   100%  209     0.2KB/s   00:00    
sm_ADA_fold3_trial20_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold3_trial21_5w_hyperparameters.csv   100%  212     0.2KB/s   00:00    
sm_ADA_fold3_trial22_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold3_trial23_5w_hyperparameters.csv   100%  206     0.2KB/s   00:00    
sm_ADA_fold3_trial24_5w_hyperparameters.csv   100%  206     0.2KB/s   00:00    
sm_ADA_fold3_trial25_5w_hyperparameters.csv   100%  211     0.2KB/s   00:00    
sm_ADA_fold3_trial2_5w_hyperparameters.csv    100%  207     0.2KB/s   00:00    
sm_ADA_fold3_trial26_5w_hyperparameters.

sm_ADA_fold4_trial21_5w_hyperparameters.csv   100%  209     0.2KB/s   00:00    
sm_ADA_fold4_trial22_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold4_trial23_5w_hyperparameters.csv   100%  204     0.2KB/s   00:00    
sm_ADA_fold4_trial24_5w_hyperparameters.csv   100%  206     0.2KB/s   00:00    
sm_ADA_fold4_trial25_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold4_trial2_5w_hyperparameters.csv    100%  208     0.2KB/s   00:00    
sm_ADA_fold4_trial26_5w_hyperparameters.csv   100%  205     0.2KB/s   00:00    
sm_ADA_fold4_trial27_5w_hyperparameters.csv   100%  209     0.2KB/s   00:00    
sm_ADA_fold4_trial28_5w_hyperparameters.csv   100%  210     0.2KB/s   00:00    
sm_ADA_fold4_trial29_5w_hyperparameters.csv   100%  207     0.2KB/s   00:00    
sm_ADA_fold4_trial30_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold4_trial31_5w_hyperparameters.csv   100%  207     0.2KB/s   00:00    
sm_ADA_fold4_trial32_5w_hyperparameters.

sm_ADA_fold5_trial2_5w_hyperparameters.csv    100%  208     0.2KB/s   00:00    
sm_ADA_fold5_trial26_5w_hyperparameters.csv   100%  206     0.2KB/s   00:00    
sm_ADA_fold5_trial27_5w_hyperparameters.csv   100%  209     0.2KB/s   00:00    
sm_ADA_fold5_trial28_5w_hyperparameters.csv   100%  210     0.2KB/s   00:00    
sm_ADA_fold5_trial29_5w_hyperparameters.csv   100%  206     0.2KB/s   00:00    
sm_ADA_fold5_trial30_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold5_trial31_5w_hyperparameters.csv   100%  206     0.2KB/s   00:00    
sm_ADA_fold5_trial32_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold5_trial33_5w_hyperparameters.csv   100%  206     0.2KB/s   00:00    
sm_ADA_fold5_trial34_5w_hyperparameters.csv   100%  207     0.2KB/s   00:00    
sm_ADA_fold5_trial35_5w_hyperparameters.csv   100%  208     0.2KB/s   00:00    
sm_ADA_fold5_trial3_5w_hyperparameters.csv    100%  206     0.2KB/s   00:00    
sm_ADA_fold5_trial36_5w_hyperparameters.

### Combine files across trials for every ligand+classifier+fold combination

In [216]:
for ligand in ligands:
    
    for classifier in classifiers:
        #for i in range(folds_num):
        for i in range(1,2):
            fold = i+1
            
            print "fold="+str(fold)
            fold_df = pd.DataFrame()
            for trial in range(no_trials):
                trial_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_trial"+str(trial)+"_"+str(folds_num)+"w_hyperparameters.csv"
                trial_hyperparameters_df = pd.read_csv(trial_filename, sep='\t', index_col=0)
                fold_df = fold_df.append(trial_hyperparameters_df)
            fold_df = fold_df.sort_values(by = "mean_AUPRC", ascending = False)
            fold_df.to_csv(output_path+ligand+"_"+classifier+"_fold"+str(fold)+"_hyperparameters.csv", sep='\t')

fold=2
